# Identification

In [ ]:
from tuto_control_lib.systems import UnknownSystem
from tuto_control_lib.plot import *

import matplotlib.pyplot as plt
import numpy as np
from math import exp, log, pi, cos
from statistics import mean

For moment, we supposed the model of the system known (i.e. the coeficients $a$ and $b$).
But in practice, we do not know them.

In this Section, we will perform what is called the *Identification* to get the coefficient of the system model.

The idea of the identification phase is simple: "Get the relation between the input and output".

To do this, the most basic way is to perform a serie of step inputs and observe the output.

In this Section, we will use the `UnknownSystem` and try to find its coefficients.

In [ ]:
system = UnknownSystem()
y_values_ident, u_values_ident, u, system, max_iter = [], [], 0, UnknownSystem(), 200
for i in range(max_iter):
    y = system.sense()
    y_values_ident.append(y)
    
    u = (i + 20) // 20
    
    system.apply(u)
    u_values_ident.append(u)

plot_u_y(u_values_ident, y_values_ident)

We are looking for an expression for a first order system of the following form:

$$
y(k+1) = a y(k) + b u(k)
$$

In steady state, the input is constant: $u_{ss}$.

Thus:

$$
y_{ss} = a y_{ss} + b u_{ss} \implies \frac{y_{ss}}{u_{ss}} = \frac{b}{1 - a}
$$

In our case:

In [ ]:
previous_u = 1
gain_ss = None
for (u, y) in zip(u_values_ident, y_values_ident):
    if u != previous_u:
        print(f"u: {previous_u} -> {gain_ss}")
        previous_u = u
    else:
        gain_ss = y / u
print(f"u: {previous_u} -> {gain_ss}")

So the static gain is around `0.85`.

We will perform a Least Mean Square to get an estimation of the model:

In [ ]:
u_values_identification = u_values_ident
y_values_identification = y_values_ident

s1 = sum(y * y for y in y_values_identification)
s2 = sum(u * y for (u, y) in zip(u_values_identification, y_values_identification))
s3 = sum(u * u for u in u_values_identification)
s4 = sum(y * z for (y, z) in zip(y_values_identification[:-2], y_values_identification[1:]))
s5 = sum(u * z for (u, z) in zip(u_values_identification[:-2], y_values_identification[1:]))

a_est = (s3 * s4 - s2 * s5) / (s1 * s3 - s2 * s2)
b_est = (s1 * s5 - s2 * s4) / (s1 * s3 - s2 * s2)

print(f"a: {a_est}, b: {b_est} -> gain: {b_est / (1 - a_est)}")

In [ ]:
max_iter = 200
system = UnknownSystem()
y_values, u_values, u, system, integral = [], [], 0, UnknownSystem(), 0
model = []
y_model = 0

for i in range(max_iter):
    y = system.sense()
    y_values.append(y)
    
    u = (i + 20) // 20
    
    system.apply(u)
    u_values.append(u)
    
    y_model = a_est * y_model + b_est * u
    model.append(y_model)

plot_model_compa(y_values, model)

# Designing a PI Controller

In [ ]:
ks = 10
mp = 0.05

r = exp(-4/ks)
theta = pi * log(r) / log(mp)

kp = (a_est - r * r) / b_est
ki = (1 - 2 * r * cos(theta) + r * r) / b_est

print(f"Kp = {kp}\nKi = {ki}")

In [ ]:
reference_value = 1
y_values, u_values, u, system, integral_error, max_iter = [], [], 0, UnknownSystem(), 0, 50

for i in range(max_iter):
    y = system.sense()
    y_values.append(y)
    
    error = reference_value - y
    integral_error += error
    u = kp * error + ki * integral_error
    
    system.apply(u)
    u_values.append(u)

plot_u_y(u_values, y_values, reference_value)

In [ ]:
e_ss = reference_value - y_values[-1]
max_overshoot = (max(y_values) - y_values[-1]) / y_values[-1]
settling_time = len([x for x in y_values if abs(x - y_values[-1]) > 0.05])

print(f"Precision: {e_ss}")
print(f"Settling Time: {settling_time} -> desired: < {ks}")
print(f"Max. Overshoot: {max_overshoot} -> desired: < {mp}")

[Back to menu](./00_Main.ipynb) or [Next chapter](./06_RealSystem.ipynb)